# CPI-SQLite Database

The purpose of this project is to load data from the BLS website, which is regularly updated in the form of text files, to a simple SQLite database. 

Information on the CPI Survey: https://download.bls.gov/pub/time.series/cu/cu.txt

## Process Outline

1. Get text from BLS website

2. Create database SCHEMA

3. Load files into DB


In [1]:
import sqlite3
import urllib.request
import pandas as pd
from io import StringIO

In [8]:
base_url = 'https://download.bls.gov/pub/time.series/cu/'
area_codes = 'cu.area'
item_codes = 'cu.item'
period_codes = 'cu.period'
data_file = 'cu.data.0.Current'

In [9]:
def read_text_from_url(url):
    """
    Reads text file from a URL.
    Ex: read_text_from_url(base_url+period_codes)
    """
    try:
        with urllib.request.urlopen(url) as response:
            data = response.read()
            text = data.decode('utf-8')
            return text
    except Exception as e:
        print(f'Error: {e}')
        return None

def text_to_df(text, separator):
    """
    Transforms text into pandas dataframe
    Ex: text_to_df(text, separator('\t'))
    """
    try:
        df = pd.read_csv(StringIO(text), sep=separator)
        return df
    except Exception as e:
        print(f'Error: {e}')
        return None

In [10]:
text = read_text_from_url(base_url+period_codes)
text_to_df(text, separator="\t")

,period,period_abbr,period_name
0,M01,JAN,January
1,M02,FEB,February
2,M03,MAR,March
3,M04,APR,April
4,M05,MAY,May
5,M06,JUN,June
6,M07,JUL,July
7,M08,AUG,August
8,M09,SEP,September
9,M10,OCT,October


In [ ]:
create_periods_query = f""" 
    CREATE TABLE IF NOT EXISTS periods (
        period TEXT,
        period_abbr TEXT,
        period_name TEXT
    )
"""

def create_table(conn, query):
    try:
        with conn:
            cursor = conn.cursor()
            cursor.execute(query)
            print('Table has been created in the database')
    except Exception as e:
        print(f'Error: {e}')


In [3]:
def create_sqlite_connection(db_file):
    """
    Creates a database connection to a SQLite database.
    If the db_file does not exist, that file is automatically created.
    """
    conn = None
    try: 
        conn = sqlite3.connect(db_file)
        # conn = sqlite3.connect(':memory:') creates the database in memory
        print(sqlite3.version)
    except sqlite3.Error as e:
        print(e)
    finally:
        if conn:
            conn.close()



In [ ]:
def create_sqlite_table_from_url(conn, url, table_name, separator):
    try:
        text = read_text_from_url(url)
        if text:
            rows = text.strip().split('\n')
            rows = [row.split(separator) for row in rows]

            column_names = rows[0]
            columns = ', '.join([f'"{col}" TEXT' for col in column_names]) 
            create_table_query = f"""
                CREATE TABLE IF NOT EXISTS {table_name} ({columns});
            """
            
            with conn:
                cursor = conn.cursor()
                cursor.execute(create_table_query)

                insert_query = f""" 
                    INSERT INTO {table_name} VALUES ({','.join('?' * len(rows[0]))})
                """
                cursor.executemany(insert_query, rows)
                print(f'Table "{table_name}" has been created in the database.')
        else:
            print("Unable to read the text file from URL.")
    except Exception as e:
        print(f'Error: {e}')
                

In [6]:
if __name__ == '__main__':
    db_file = 'sqlite-test.db'  
    conn = create_sqlite_connection('sqlite-test.db')
    if conn:
        table_name = 'periods'
        url = base_url+period_codes
        create_sqlite_table_from_url(conn, url, table_name)
        conn.close()

2.6.0
